In [4]:
import pandas as pd
import numpy as np

server_data = pd.read_csv('Analyst Testcase_Serverdata v2.csv')
server_data

,,,Timestamp;Servername;Metric;Unit;Average;Max;Min;MemoryMB;NumCpu;CpuMHz;Cluster;
9/22/14;ITLT_C1_S001;mem.active.average;KB;190856,3838;611107,65;41101,2;4096;2;5320;1;
9/22/14;ITLT_C1_S001;cpu.usagemhz.average;MHz;46,8072;303,6;25,25;4096;2;5320;1;
9/23/14;ITLT_C1_S001;mem.active.average;KB;179302,9824;661132,24;62408,58;4096;2;5320;1;
9/23/14;ITLT_C1_S001;cpu.usagemhz.average;MHz;50,0955;300,84;23,04;4096;2;5320;1;
9/24/14;ITLT_C1_S001;mem.active.average;KB;208730,819;786261,83;54718,75;4096;2;5320;1;
9/24/14;ITLT_C1_S001;cpu.usagemhz.average;MHz;45,6967;350,86;26,16;4096;2;5320;1;
9/25/14;ITLT_C1_S001;mem.active.average;KB;180816,3721;894138,3;46805,04;4096;2;5320;1;
9/25/14;ITLT_C1_S001;cpu.usagemhz.average;MHz;50,6412;339,36;22,5;4096;2;5320;1;
9/26/14;ITLT_C1_S001;mem.active.average;KB;190580,0035;910945,49;42778,8;4096;2;5320;1;
9/26/14;ITLT_C1_S001;cpu.usagemhz.average;MHz;47,6805;455,4;25,3;4096;2;5320;1;
